In [1]:
import os
import pandas as pd

from feature_engineering import *
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
DATA_PATH = '../HomeCreditDefaultRisk/Data'

In [4]:
tables = [pd.read_csv(os.path.join(DATA_PATH, table)) for table in os.listdir(DATA_PATH) if table.endswith('csv.zip')]
ins, prev, bureau, bb, df, cc, test_df, pos = tables

In [5]:
# EXECUTE FE

limit = None

df = application_train_test(df, test_df, nan_as_category = False, limit=limit)
bureau_agg = bureau_and_balance(bureau, bb, nan_as_category = True, limit=limit)
prev_agg = previous_applications(prev, nan_as_category = True, limit=limit)
pos_agg = pos_cash(pos, nan_as_category = True, limit=limit)
ins_agg = installments_payments(ins, nan_as_category = True, limit=limit)
cc_agg = credit_card_balance(cc, nan_as_category = True, limit=limit)

In [ ]:
B2o8b1d2e1c

In [ ]:
pos_agg = pos_agg.reset_index()
#ins_agg = ins_agg.reset_index()
cc_agg = cc_agg.reset_index()

In [ ]:
# MERGE ALL

def merge(df,bureau_agg,prev_agg,pos_agg,cc_agg):
    
    df = (df
           .merge(bureau_agg, how='left', on='SK_ID_CURR')
           .merge(prev_agg, how='left', on='SK_ID_CURR')
           .merge(pos_agg, how='left', on='SK_ID_CURR')
           .merge(ins_agg, how='left', on='SK_ID_CURR')
           .merge(cc_agg, how='left', on='SK_ID_CURR')
          )
    
    del bureau_agg
    del prev_agg
    del pos_agg
    del ins_agg
    del cc_agg
    
    return df

In [ ]:
df = merge(df, bureau_agg, prev_agg, pos_agg, cc_agg)

In [ ]:
# SHARE MISSING VALUES

df['SHARE_NAN'] = df.isnull().sum(axis=1) / df.shape[1]

In [ ]:
# SPLIT TRAIN AND TEST

train_df = df[df['TARGET'].notnull()]
test_df = df[df['TARGET'].isnull()]
del df

In [ ]:
# FILLNA BY A SINGLE VALUE TO GIVE A SIGNAL

train_df.replace([np.inf, -np.inf], np.nan, inplace=True)
test_df.replace([np.inf, -np.inf], np.nan, inplace=True)

train_df.fillna(-1, inplace=True)
test_df.fillna(-1, inplace=True)

assert train_df.isnull().sum().sum() == test_df.isnull().sum().sum() == 0